In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
def min_max_value():
  df = pd.read_csv('/content/drive/MyDrive/train.csv')

  value_columes = ["배추_가격(원/kg)","무_가격(원/kg)", "양파_가격(원/kg)", "건고추_가격(원/kg)", "마늘_가격(원/kg)",\
          "대파_가격(원/kg)", "얼갈이배추_가격(원/kg)", "양배추_가격(원/kg)",\
          "깻잎_가격(원/kg)", "시금치_가격(원/kg)", "미나리_가격(원/kg)",\
          "당근_가격(원/kg)",  "파프리카_가격(원/kg)", "새송이_가격(원/kg)", "팽이버섯_가격(원/kg)", "토마토_가격(원/kg)",\
          "청상추_가격(원/kg)","백다다기_가격(원/kg)", "애호박_가격(원/kg)",\
          "캠벨얼리_가격(원/kg)", "샤인마스캇_가격(원/kg)"]

  for i in value_columes:
    df = df[df[i] !=0 ]

  max_value = []
  min_value = []

  for value in value_columes:
    min_ = min(df[value])
    min_value.append(min_)
    max_ = max(df[value])
    max_value.append(max_)

  return min_value, max_value


min_value, max_value = min_max_value()

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/train.csv')

value_columes = ["배추_가격(원/kg)","무_가격(원/kg)", "양파_가격(원/kg)", "건고추_가격(원/kg)", "마늘_가격(원/kg)",\
          "대파_가격(원/kg)", "얼갈이배추_가격(원/kg)", "양배추_가격(원/kg)",\
          "깻잎_가격(원/kg)", "시금치_가격(원/kg)", "미나리_가격(원/kg)",\
          "당근_가격(원/kg)",  "파프리카_가격(원/kg)", "새송이_가격(원/kg)", "팽이버섯_가격(원/kg)", "토마토_가격(원/kg)",\
          "청상추_가격(원/kg)","백다다기_가격(원/kg)", "애호박_가격(원/kg)",\
          "캠벨얼리_가격(원/kg)", "샤인마스캇_가격(원/kg)"]

for i in value_columes:
  df = df[df[i] !=0 ]
  if i == "건고추_가격(원/kg)":
    df = df[df[i] <= 30000]
    df = df[df[i] >= 1000] 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_columes = ["배추_거래량(kg)", "배추_가격(원/kg)","무_거래량(kg)", "무_가격(원/kg)","양파_거래량(kg)", "양파_가격(원/kg)","건고추_거래량(kg)", "건고추_가격(원/kg)","마늘_거래량(kg)", "마늘_가격(원/kg)",\
           "대파_거래량(kg)", "대파_가격(원/kg)","얼갈이배추_거래량(kg)", "얼갈이배추_가격(원/kg)","양배추_거래량(kg)", "양배추_가격(원/kg)",\
           "깻잎_거래량(kg)", "깻잎_가격(원/kg)","시금치_거래량(kg)", "시금치_가격(원/kg)","미나리_거래량(kg)", "미나리_가격(원/kg)",\
           "당근_거래량(kg)", "당근_가격(원/kg)",  "파프리카_거래량(kg)", "파프리카_가격(원/kg)","새송이_거래량(kg)", "새송이_가격(원/kg)","팽이버섯_거래량(kg)", "팽이버섯_가격(원/kg)", "토마토_거래량(kg)", "토마토_가격(원/kg)",\
           "청상추_거래량(kg)", "청상추_가격(원/kg)","백다다기_거래량(kg)", "백다다기_가격(원/kg)","애호박_거래량(kg)", "애호박_가격(원/kg)",\
           "캠벨얼리_거래량(kg)", "캠벨얼리_가격(원/kg)","샤인마스캇_거래량(kg)", "샤인마스캇_가격(원/kg)"]


scaled = scaler.fit_transform(df[scale_columes])
df_ = pd.DataFrame(scaled, columns= scale_columes )


from sklearn.model_selection import train_test_split
train, valid = train_test_split(df_, test_size=0.3, random_state= 0, shuffle=True)
valid, test = train_test_split(valid, test_size=0.4, random_state=0, shuffle=False)

print("----------------train, test 분리 후 shape 출력-------------------")
print("train : ", train.shape,"validation :", valid.shape)

def windowed_dataset(series, window_size, batch_size, shuffle, asp):
    #series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + asp, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + asp))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-asp], w[-1][1::2])) # two, four
    return ds.batch(batch_size).prefetch(1)

WINDOW_SIZE= 50
BATCH_SIZE= 10 # 데이터는 총 50개씩 랜덤으로 뽑음
asp = 7
train_data = windowed_dataset(train, WINDOW_SIZE, BATCH_SIZE, True, asp) 
valid_data = windowed_dataset(valid, WINDOW_SIZE, BATCH_SIZE, False, asp) 
test_data = windowed_dataset(test, WINDOW_SIZE, BATCH_SIZE, False, asp)

for data in train_data.take(1):
    print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
    print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')


----------------train, test 분리 후 shape 출력-------------------
train :  (623, 42) validation : (160, 42)
데이터셋(X) 구성(batch_size, window_size, feature갯수): (10, 50, 42)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (10, 21)


In [ ]:
# Global 데이터를 포함 할 경우 아닐 경우 # 

# 1. locals_context : 포함 안함
# 2. locals_context : 포함함

# 큰 특징 : global data를 포함하는 경우 총 sector의 수가 달라짐
# ex) 입력 : 21개의 sector
#     출력 : 20개의 sector

# local_context class의 sector_num만 수정한다면 자동으로 바뀜


In [ ]:
class Context_Vector(tf.keras.layers.Layer):
  def __init__(self):
    super(Context_Vector, self).__init__()
    self.FC1 = tf.keras.layers.Dense(2) 
    self.LSTM = tf.keras.layers.LSTM(128, return_sequences = True, return_state = True, dropout= 0.2) 
    self.LayerNorm = tf.keras.layers.LayerNormalization (axis = -1) # test 

  def Attention(self, query, key, value): # query : LSTM hidden, key, 
    scores = tf.matmul(query, key, transpose_b=True) # score
    distribution = tf.nn.softmax(scores)
    return tf.matmul(distribution, value)

  def call(self, input): # batch, window_size, feature
    FC1 = self.FC1(input)
    LSTM_1, LSTM_hidden, LSTM_cell = self.LSTM(FC1) # batch, window_size, feature
    #print("LSTM_output : ",LSTM_1) # batch, window_size, LSTM units
    LSTM_hidden = tf.expand_dims(LSTM_hidden, 1)
    #print("LSTM_hidden : ",LSTM_hidden)
    Attention1 = self.Attention(LSTM_hidden, LSTM_1, LSTM_1) 
    #print("Attention output : ",Attention1)
    LayerNorm = self.LayerNorm(Attention1)
    return LayerNorm # (batch, 1, 128)
 
class locals_context(tf.keras.layers.Layer):

  def __init__(self,sector_num):
    self.sector_num = sector_num
    super(locals_context, self).__init__()
    self.context_layers = [Context_Vector()
                       for _ in range(sector_num)]   

  def call(self, inputs):
    inputs = self.preprocess(inputs)  #batch, sector_size, window_size, feature#

    # LSTM input reshape #
    # 기존 : batch, sector_size, window_size, feature
    # 각 sector에 따라서, for문 반복
    # 변경 : batch, window_size, feature
    context_lstm = []
    for sector_num in range(self.sector_num):
      lstm_input1 = inputs[:,sector_num,:,:] # batch, window_size, feature

      att_lstm = self.context_layers[sector_num](lstm_input1) # batch, 1, LSTM units
      #att_lstm = tf.squeeze(att_lstm) # batch, LSTM units
      context_lstm.append(att_lstm)

    context = tf.stack(context_lstm)
    context = tf.squeeze(context,2)
    # temp shape =  sector_num, batch_size, LSTM units #
    # 변경 -> temp shape = batch_size, sector_num, LSTM units # 
    after_att_lstm = tf.transpose(context, perm= [1, 0, 2]) # batch_size, sector_num, LSTM units # 
    return after_att_lstm




  def preprocess(self, inputs):
    
    #  데이터의 전처리 과정  #
    # input shape = (batch, window_size, feature (concat 되어있음) ) 
    # output shape = (batch, sector_num, winodw_size, feature)
    
    test_input = inputs
    thrid_sizes = []


    batchs = inputs.shape[0]
    windows = inputs.shape[1]
    for batch in range(batchs):

      start = 0 # feature의 시작점
      end = 2 # feature의 종료점 
      sec_sizes = []     
      for num_of_sector in range(self.sector_num): # batch
        temp = []
        for i in range(windows):  
          reshaped_data = test_input[batch][i][start:end]
          reshaped_data = tf.reshape(reshaped_data, [1,2])
          temp.append(reshaped_data)  
        first_size = tf.stack(temp) # 50, 1, 2
        first_size = tf.reshape(first_size, [windows,2])
        sec_sizes.append(first_size)
        start += 2
        end += 2
      print("*", end = "" )
      # 반복 batch만큼 반복시켜서, 새로운 데이터 shape을 만듬
      #sec_sizes =  sector_size, window_size, feature
      sec_sizes = tf.stack(sec_sizes)
      thrid_sizes.append(sec_sizes)

    thrid_sizes = tf.stack(thrid_sizes)
    reshaped_output = thrid_sizes
    return reshaped_output  #batch, sector_size, window_size, feature#

#     2. multi_context를 사용한 상황 (입력 처음 sector가 global context로 사용)     #


# class locals_context(tf.keras.Model):
#   def __init__(self,sector_num, beta):
#     self.sector_num = sector_num
#     super(locals_context, self).__init__()
#     self.context_layers = [Context_Vector()
#                        for _ in range(sector_num)]   
#     self.multi = tf.TensorArray(tf.float32, size = 0, dynamic_size=True)
#     self.multis = tf.TensorArray(tf.float32, size = 0, dynamic_size=True)
#     self.beta = beta


#   def call(self, inputs):

#     test_input = inputs

#     batch_len = tf.shape(test_input)[0] # Batch_size
    
#     for batch in range(batch_len):
#       start = 0 # 데이터의 간격
#       end = 2
#       for num_of_sector in range(self.sector_num):
#         temp = tf.TensorArray(tf.float32, size = 0, dynamic_size=True)
#         for i in range(50):  
#           reshaped_data = test_input[batch][i][start:end]
#           reshaped_data = tf.reshape(reshaped_data, [1,2])
#           temp = temp.write(i, reshaped_data)
          
#         context_input = temp.stack() # 50,2
#         context_input = tf.reshape(context_input, [1,50,2])

#         if num_of_sector == 0:
#           global_context = self.context_layers[num_of_sector](context_input)
#           continue
#         local_context = self.context_layers[num_of_sector](context_input)
#         multi_context = local_context + self.beta * global_context
        
#         self.multi = self.multi.write(num_of_sector-1, multi_context) # multi_context의 경우

#         start += 2
#         end += 2
#       output = tf.reshape(tensor = self.multi.stack(), shape = [self.sector_num-1,128]) # 1, sector-1 개 , 128
#       self.multis = self.multis.write(batch, output) # BACTH_SIZE, OUTPUT
#     output1 = self.multis.stack()
#     return output1

class MultiHeadAttention(tf.keras.layers.Layer): # 
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q):

    batch_size = tf.shape(q)[0]
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)


    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
    
    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
    
    return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

def scaled_dot_product_attention(q, k, v): # transformer encoder 일부 수정
  matmul_qk = tf.matmul(q, k, transpose_b=True)
  dk = tf.cast(tf.shape(k)[-1], tf.float32) # d_model 차원을 알기위해
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk) # scaled 시킴
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
  output = tf.matmul(attention_weights, v)
  return output, attention_weights

class EncoderLayer(tf.keras.layers.Layer):   # batch, sector_size, lstm_units#
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(axis = 1)
    self.layernorm2 = tf.keras.layers.LayerNormalization(axis = 1)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training):
    
    attn_output, _ = self.mha(x, x, x)  # batch, sector_size, lstm_units#
    #---------------------------------------------------------------------#
    attn_output = self.dropout1(attn_output,training = training)
    out1 = self.layernorm1(x + attn_output)  #  batch, sector_size, lstm_units#
    print(out1)
    ffn_output = self.ffn(out1)  #  batch, sector_size, lstm_units#
    ffn_output = self.dropout2(ffn_output,training = training) # training 할 때 배우기
    out2 = self.layernorm2(out1 + ffn_output)  # batch, sector_size, lstm_units#
    return out2 

class Linear_Mapping(tf.keras.layers.Layer): #  batch, sector_size, lstm_units#
  def __init__(self, units, sector_num):
    super(Linear_Mapping, self).__init__()
    self.sector_num = sector_num
    self.W1 = tf.keras.layers.Dense(units, activation = 'relu')
    self.dropout = tf.keras.layers.Dropout(0.2)
    self.W2 = tf.keras.layers.Dense(20)
    self.W3 = tf.keras.layers.Dense(1)

  def call(self, input, training):
    batch = input.shape[0]
    linear_output = self.W1(input)
    dropout3 = self.dropout(linear_output, training = training)
    linear_output = self.W2(dropout3)
    linear_output = self.W3(linear_output)
    return tf.reshape(linear_output, [batch, self.sector_num] ) # Batch size, sector_size

class predict_value(tf.keras.Model):
  def __init__(self, SECTOR_NUM ,LSTM_UNITS, NUM_HEAD, FF, LINEAR_UNIT):
    super(predict_value, self).__init__()
    self.localsa = locals_context(SECTOR_NUM) # sector_num( Global 포함)
    self.encoder = EncoderLayer(LSTM_UNITS, NUM_HEAD, FF)
    self.linear_mapping = Linear_Mapping(LINEAR_UNIT,SECTOR_NUM)

  def call(self, inputs, training):
    local = self.localsa(inputs)
    encoder = self.encoder(local, training)
    linear_out = self.linear_mapping(encoder, training)
    return linear_out


In [ ]:
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

sample_predict_value = predict_value(
    SECTOR_NUM= 21,
    LSTM_UNITS = 128, 
    NUM_HEAD = 4, 
    FF=128, 
    LINEAR_UNIT = 40,
    )

sample_predict_value(tf.keras.layers.Input(shape = (50,42), batch_size= 10))

sample_predict_value.summary()

**********Tensor("predict_value/encoder_layer/layer_normalization_21/batchnorm/add_1:0", shape=(10, 21, 128), dtype=float32)
Model: "predict_value"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 locals_context (locals_cont  multiple                 1414014   
 ext)                                                            
                                                                 
 encoder_layer (EncoderLayer  multiple                 99156     
 )                                                               
                                                                 
 linear__mapping (Linear_Map  multiple                 6001      
 ping)                                                           
                                                                 
Total params: 1,519,171
Trainable params: 1,519,171
Non-trainable params: 0
__________________________________________________

In [ ]:

train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')

optimizer = tf.keras.optimizers.Adam(0.001) # 0.005, 0.001, 0.0005, 0.0001 
loss_object = tf.keras.losses.MeanSquaredError()

checkpoint_path = "/content/drive/MyDrive/NewVersion/checkpoint3.ckpt"

ckpt = tf.train.Checkpoint(model = sample_predict_value,
                            optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep= 1)



@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = sample_predict_value(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, sample_predict_value.trainable_variables)
  optimizer.apply_gradients(zip(gradients, sample_predict_value.trainable_variables))

  train_loss(loss)

@tf.function
def test_step(images, labels):
  predictions = sample_predict_value(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)

EPOCHS = 150

min = 1

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()

  for train_x, train_y in train_data:
    if train_x.shape[0] != BATCH_SIZE:
      continue
    train_step(train_x, train_y)

  for valid_x, valid_y in valid_data:
    if valid_x.shape[0] != BATCH_SIZE:
      continue
    test_step(valid_x, valid_y)

  print( f'Epoch {epoch + 1}, '
  f'Train Loss: {train_loss.result()}, '
  f'validation Loss: {test_loss.result()},\n'
  )

  # validaiton loss value is the lowest
  if test_loss.result() < min:
    min = test_loss.result()
    ckpt_save_path = ckpt_manager.save()
    print("Validation Loss is better than Train Loss. so New parameter is stored drive \n")


**********Tensor("predict_value/encoder_layer/layer_normalization_21/batchnorm/add_1:0", shape=(10, 21, 128), dtype=float32)
**********Tensor("predict_value/encoder_layer/layer_normalization_21/batchnorm/add_1:0", shape=(10, 21, 128), dtype=float32)
**********Tensor("predict_value/encoder_layer/layer_normalization_21/batchnorm/add_1:0", shape=(10, 21, 128), dtype=float32)
Epoch 1, Train Loss: 0.3207542598247528, validation Loss: 0.04008394479751587,

Validation Loss is better than Train Loss. so New parameter is stored drive 

Epoch 2, Train Loss: 0.07072760909795761, validation Loss: 0.03756430000066757,

Validation Loss is better than Train Loss. so New parameter is stored drive 

Epoch 3, Train Loss: 0.0537097342312336, validation Loss: 0.03639240562915802,

Validation Loss is better than Train Loss. so New parameter is stored drive 

Epoch 4, Train Loss: 0.047792453318834305, validation Loss: 0.034931693226099014,

Validation Loss is better than Train Loss. so New parameter is stor

In [ ]:
test_input = df_[-51:-1]
test_input = np.expand_dims(test_input,0)

results = sample_predict_value(test_input, training = False)

if ckpt_manager.latest_checkpoint:
   ckpt.restore(ckpt_manager.latest_checkpoint)
   print('Latest checkpoint restored!!')

final_result = []

sector_num = 21
for i in range(sector_num):
  value = results[0][i] * (max_value[i] - min_value[i]) + min_value[i]
  final_result.append(value)
final_result = tf.stack(final_result)

print("\n21가지의 농산물 가격 출력 : ")
print(final_result)



*tf.Tensor(
[[[ 9.5077306e-01 -4.2049289e-02  1.5824772e-01 ... -1.2339503e-03
    1.9781667e-01 -1.2552534e-01]
  [ 3.2637286e-01 -1.7531331e-01 -8.4314696e-02 ... -3.7118942e-01
   -1.8052176e-01 -3.8221294e-01]
  [-6.7662132e-01 -2.2568813e-01  3.9455861e-02 ...  1.5022025e-01
   -1.2660500e-01 -6.8805826e-01]
  ...
  [-1.5165542e-01  2.9242006e-01  6.0833389e-01 ... -5.6306183e-02
    1.7253625e-01  2.9382354e-01]
  [-3.4020257e-01 -5.2597690e-01  1.3681761e+00 ... -4.9871132e-02
    3.7098601e-01  9.4004834e-01]
  [-5.7968831e-01  1.8478721e-01  2.0294459e-01 ...  1.7208068e-01
   -1.2573123e-02 -9.1147497e-02]]], shape=(1, 21, 128), dtype=float32)
Latest checkpoint restored!!

21가지의 농산물 가격 출력 : 
tf.Tensor(
[  716.52045   567.275     676.8801  29093.34     3771.6755   1291.8198
  1142.1919    607.65216  5572.424    3172.8826   2751.5264   1057.5107
  3259.2314   2139.0303   1729.8989   2235.5776   4077.2935   1908.4614
  1786.4315   4062.7415  12850.806  ], shape=(21,), dtype=floa